EPI Notebook
============
**Note: this code will not run.
See /cubric/data/c24073803/pypulseq_repo/pypulseq/examples/scripts/write_epi.py for runnable script**

## Objectives
1. To decontruct and understand the lines of code within write_epi.py
2. To alter parameters and explore their resultant effect
3. To explore changing the shape of the RF pulse





**Rik Khot**\
**PhD Student**\
**Cardiff University School of Physics & Astronomy**\
**Supervisors: Kevin Murphy, Ian Driver, Emre Kopanoglu**

In [ ]:
#Import packages
import numpy as np
import pypulseq as pp


For information on the specific functions that are part of the pypulseq package [click this link](https://pypulseq.readthedocs.io/en/master/pypulseq.html).

### MAIN FUNCTION
The function main() contains all the code required to produce a .seq file.\
.seq files are what the MRI scanner reads to initiate a pulse sequence. \
The main() code also produces plots, which we will discuss further on.

In [ ]:
def main(plot: bool = False, write_seq: bool = False, seq_filename: str = 'epi_pypulseq.seq'):

Let us consider the variables:
- plot
    - Variable of Boolean type (true/false)
- write_seq
    - Variable of Boolean type (true/false)
- seq_filename
    - Variable of string type 
    - Variable hardcoded as "epi_pypulseq.seq"

In [ ]:
# Define FOV and resolution
    fov = 220e-3
    Nx = 64
    Ny = 64
    slice_thickness = 3e-3  # Slice thickness
    n_slices = 3

N.B. We will assume that fov and slice thickness is quoted in **meters** 

The 1st section of the function hardcodes the values for:
1. Field of view (FOV)
    - *distance over which the MR image is acquired*
    - *FOV and pixel width determine how many digitised samples are required to construct an image with required resolution*
    - Spacing between k-space samples, $\Delta k = \frac{1}{FOV}$
2. Matrix size
    - relates to the number of pixels within an image
    - affects resolution and image quality
    would expect 128 x 128 or 256 x 256
3. Slice Thickness
    - Based on RF bandwidth
    - In general(NOT MR SPECIFIC), RF wavelength range: $~3mm - 300,000m$

4. No. of slices
    - $n_{maxslices} = \frac{TR}{TE}$

In [ ]:
# Set system limits
    system = pp.Opts(
        max_grad=32,
        grad_unit='mT/m',
        max_slew=130,
        slew_unit='T/m/s',
        rf_ringdown_time=30e-6,
        rf_dead_time=100e-6,
    )


pp.Opts() function passes the following variables (see documentation for more details).
- adc dead time (default = 0)
    - time taken to digitise continuous waveforms recieved

- gyromagnetic ratio (default value, proton)

- gradient raster time (default value)
    - *rasterization: converting continuous wave into discrete points*
    - *Grad raster time: time taken to "digitise" gradient waveforms*

- gradiant unit (default Hz/m)

- maximum gradient (default 0)

- maximum slew rate (default 0)
    - "quickest" gradient can ramp up to it's maximum amplitude

- RF dead time
    - when rf coil switches from transmit to recieve
    - cannot send or recieve data
    - *do we have only 1 rf coil?*

- RF raster time
    - time taken to digitise continuous RF pulses

- RF ringdown time
    - time taken for RF coil to chill out after emitting a pulse
    - to avoid interference/artefacts as much as possible

- rise time
    - time taken for gradient to ramp up to it's maximum amplitude

- slew unit (user specified, eg. T/m/s)


In [ ]:
 seq = pp.Sequence(system)  # Create a new sequence object

^^^^
Will be relevent later in sequence construction section

#### Creating Events

In [ ]:
# Create 90 degree slice selection pulse and gradient
    rf, gz, _ = pp.make_sinc_pulse(
        flip_angle=np.pi / 2,
        system=system,
        duration=3e-3,
        slice_thickness=slice_thickness,
        apodization=0.5,
        time_bw_product=4,
        return_gz=True,
        delay=system.rf_dead_time,
    )

returns rf sinc pulse and slice select trapezoidal gradient (iff slice thickness defined)

Consider make_sinc_pulse() variables:

- flip angle in radians 
- apodization (default =0)
    - I think this reduces spectral contamination artefacts?
- centre_pos
    - position of peak (assumed midway?)
- delay (in seconds)
    - usually rf dead time
- duration (default = 0.004s)
- dwell? (default 0)
- freq_offset
- max_grad
    - maximum gradient for slice select waveform (trapezioid)
- max slew
    - of slice select gradient
- phase offset? Read on phase-offset multiplanar volume imaging [click this link](https://pubmed.ncbi.nlm.nih.gov/1790368/)
- return_delay (does a delay need to be returned)
- return_gz (does slice select gradient need to be returned)
- slice_thickness (default 0)
    -**pre-defined earlier in code during FOV and resolution**
- system
    - **defined earlier in code when system limits were set**
- time_bw_product (default 4)
- use (options include "excitation", "refocusing" or "inversion")





##### Defining gradients and ADC events

In [ ]:
 # Define other gradients and ADC events
    delta_k = 1 / fov
    k_width = Nx * delta_k
    dwell_time = 4e-6
    readout_time = Nx * dwell_time
    flat_time = np.ceil(readout_time * 1e5) * 1e-5  # round-up to the gradient raster
    gx = pp.make_trapezoid(
        channel='x',
        system=system,
        amplitude=k_width / readout_time,
        flat_time=flat_time,
    )
    adc = pp.make_adc(
        num_samples=Nx,
        duration=readout_time,
        delay=gx.rise_time + flat_time / 2 - (readout_time - dwell_time) / 2,
    )


**REMEMBER $N_{x}$ and $N_{y}$ are the matrix sizes defined near line 1**
1. $\Delta k = \frac{1}{FOV}$
    - spacing between data points in k-space
2. $k_{width} = k_{FOV} = \frac{N_{x}}{FOV}$
3. Dwell time = ADC dead time after sampling
4. readout time = time taken to collect data points for single phase encoding line
5. flat time
    - np.ceil essentially rounds the value up to the nearest integer
    - don't really understand this equation
    - flat time is a specific property of fast-imaging sequences

    

# References

mriquestions.com (fov, matrix size and resolution)
pypulseq documentation